In [1]:
import pandas as pd
from pandas_profiling import ProfileReport
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import classification_report, f1_score
from sklearn import set_config
from sklearn import datasets
from sklearn import ensemble, neighbors,linear_model,svm
import eli5
set_config(display='diagram')
datasets.load*?

/data/ml/ubuntu_env/lib/python3.6/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/data/ml/ubuntu_env/lib/python3.6/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


datasets.load_boston
datasets.load_breast_cancer
datasets.load_diabetes
datasets.load_digits
datasets.load_files
datasets.load_iris
datasets.load_linnerud
datasets.load_sample_image
datasets.load_sample_images
datasets.load_svmlight_file
datasets.load_svmlight_files
datasets.load_wine

In [2]:
def sklearn_to_df(sklearn_dataset):
    df = pd.DataFrame(sklearn_dataset.data, columns=sklearn_dataset.feature_names)
    df['target'] = pd.Series(sklearn_dataset.target)
    return df

In [3]:
df = sklearn_to_df(datasets.load_breast_cancer())

numeric_features = df.select_dtypes(include=['int64', 'float64']).drop(['target'], axis=1).columns
categorical_features = df.select_dtypes(include=['object']).columns

X = df.drop('target',axis = 'columns')
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state =1)

In [4]:
#ProfileReport(df)

In [5]:
# train_values = pd.read_csv('train_values.csv')
# train_labels = pd.read_csv('train_labels.csv')
# train_data = train_values.merge(train_labels, left_on='building_id', right_on='building_id')

# train_data = train_data.drop('building_id', axis=1)
# numeric_features = train_data.select_dtypes(include=['int64', 'float64']).drop(['damage_grade'], axis=1).columns
# categorical_features = train_data.select_dtypes(include=['object']).columns
# X = train_data.drop('damage_grade', axis=1)
# y = train_data['damage_grade']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,StandardScaler,PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn import ensemble, neighbors,linear_model,svm
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score

numeric_transformer = Pipeline(steps=[
    #('feature_selection',SelectFromModel(svm.LinearSVC(penalty = 'l1',dual = False))),
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('one_hot', OneHotEncoder()),
    ('poly', PolynomialFeatures(degree = 2,interaction_only = True))
    ])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])
pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model',  ensemble.RandomForestRegressor(max_depth = 3))])
    
#model = pipe.fit(X_train, y_train)
pipe

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'tex...
       'worst smoothness', 'worst compactness', 'worst concavity',
       'worst concave points', 'worst symmetry', 'worst fractal dimension'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('one_hot',
                                                                   OneHotEncoder()),
                                                                  ('poly',
                                                                   PolynomialFeatures(interaction_only=True))]),
                                                  Index([], dtype='object'))])),
                ('model', RandomForestRegressor(max_depth=3))])

In [9]:
try:
    pipe = Pipeline(steps=[('preprocessor', preprocessor),('model',  ensemble.RandomForestRegressor(max_depth = 3))])
    pipe.fit(X_train,y_train)
except:
    pipe = Pipeline(steps=[('preprocessor', preprocessor),('model',  ensemble.RandomForestClassifier(max_depth = 3))])
    pipe.fit(X_train,y_train)

try:
    onehot_columns = list(pipe.named_steps['preprocessor'].named_transformers_['cat'].named_steps['one_hot']
                          .get_feature_names(input_features=categorical_features))
except:
    onehot_columns = []

numeric_features_list = list(numeric_features)
numeric_features_list.extend(onehot_columns)
eli5.explain_weights(pipe.named_steps['model'], top=10, feature_names=numeric_features_list)

Weight,Feature
0.5900 ± 0.7372,worst perimeter
0.2020 ± 0.6663,worst concave points
0.0386 ± 0.2770,worst area
0.0381 ± 0.2740,mean concave points
0.0339 ± 0.0878,worst texture
0.0240 ± 0.2340,worst radius
0.0194 ± 0.0751,mean texture
0.0100 ± 0.0560,mean concavity
0.0060 ± 0.0290,worst smoothness
0.0058 ± 0.0263,worst compactness


In [10]:
model_list = [#regressors
                linear_model.LinearRegression(),linear_model.Lasso(),linear_model.Ridge(),
              
              #ensembling - regressors
                ensemble.RandomForestRegressor(max_depth = 3),ensemble.AdaBoostRegressor(),ensemble.GradientBoostingRegressor(),
    
              #classifiers
                linear_model.LogisticRegression(max_iter = 500), neighbors.KNeighborsClassifier(),svm.SVC(),             
    
              #ensembling - classifiers
                ensemble.RandomForestClassifier(n_estimators=500,max_depth = 3),ensemble.AdaBoostClassifier(learning_rate=0.5),ensemble.GradientBoostingClassifier()

              ]

In [11]:
rejected_models = []
results = []
print('{:30s}Train  /  Test   /  Cross Validation'.format('Model'))
print('-----------------------------------------------------------')
for model in model_list:
    #pipe = make_pipeline(preprocessor,model)
    #pipe = make_pipeline(model)
    pipe                 = Pipeline(steps=[('preprocessor', preprocessor),('model', model)])
    
    try:        
        pipe.fit(X_train,y_train)
        
        model_name       = type(model).__name__
        train_results    = np.abs(pipe.score(X_train,y_train))
        test_results     = np.abs(pipe.score(X_test,y_test))
        cross_val_scores = np.abs(cross_val_score(pipe, X, y, cv=5)) 
        cval_score       = cross_val_scores.mean()
        
        results.append([model_name, train_results, test_results,cval_score])
        print('{:30s}{:.4f} / {:.4f}  /  {:.4f} '.format(model_name, train_results, test_results,cval_score))

    except:
        rejected_models.append(type(model).__name__)

df_results = pd.DataFrame(results, columns=["Model", "Train Results","Test Results",'Test Cross Validation Results'])
#print('Rejected Models : {}'.format(rejected_models))

Model                         Train  /  Test   /  Cross Validation
-----------------------------------------------------------
LinearRegression              0.7798 / 0.7394  /  0.7047 
Lasso                         0.0000 / 0.0002  /  0.1287 
Ridge                         0.7768 / 0.7412  /  0.7092 
RandomForestRegressor         0.9493 / 0.8007  /  0.8027 
AdaBoostRegressor             0.9960 / 0.8652  /  0.8895 
GradientBoostingRegressor     0.9979 / 0.8268  /  0.8042 
LogisticRegression            0.9899 / 0.9708  /  0.9807 
KNeighborsClassifier          0.9799 / 0.9591  /  0.9649 
SVC                           0.9899 / 0.9708  /  0.9736 
RandomForestClassifier        0.9874 / 0.9415  /  0.9526 
AdaBoostClassifier            1.0000 / 0.9474  /  0.9684 
GradientBoostingClassifier    1.0000 / 0.9649  /  0.9579 


In [12]:
if rejected_models == []:
    print('No models rejected')
else:    
    print('Rejected Models list: ')
    for rej_models in rejected_models:
        print('  - ',rej_models)

No models rejected


In [13]:
df_results.sort_values(by = 'Test Results',ascending = False)

,Model,Train Results,Test Results,Test Cross Validation Results
6,LogisticRegression,0.989950,0.970760,0.980686
8,SVC,0.989950,0.970760,0.973638
11,GradientBoostingClassifier,1.000000,0.964912,0.957864
7,KNeighborsClassifier,0.979899,0.959064,0.964850
10,AdaBoostClassifier,1.000000,0.947368,0.968390
9,RandomForestClassifier,0.987437,0.941520,0.952569
4,AdaBoostRegressor,0.995993,0.865223,0.889503
5,GradientBoostingRegressor,0.997903,0.826846,0.804228
3,RandomForestRegressor,0.949320,0.800709,0.802692
2,Ridge,0.776811,0.741223,0.709170
